In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]━━━ 1/2 [pymongo]


In [4]:
!pip install mongomock

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [mongomock]


In [7]:
import mongomock

from pymongo import MongoClient
from bson.objectid import ObjectId

# 1. Connect to the MongoDB Client
client = mongomock.MongoClient();

# 2. Create/Select a Database and Collection
db = client["demo_database"]
users = db["users_collection"]

print("--- 1. Create (Insert) ---")
# Insert one document
user_data = {"name": "Alice", "age": 28, "skills": ["Python", "MongoDB"]}
result = users.insert_one(user_data)
print(f"Inserted document ID: {result.inserted_id}")

# Insert many documents
multiple_users = [
    {"name": "Bob", "age": 32, "skills": ["Java"]},
    {"name": "Charlie", "age": 25, "skills": ["Python", "JS"]}
]
users.insert_many(multiple_users)

print("\n--- 2. Read (Find) ---")
# Find a single document
alice = users.find_one({"name": "Alice"})
print(f"Found Alice: {alice}")

# Find multiple documents using a query (e.g., age > 26)
print("Users older than 26:")
for user in users.find({"age": {"$gt": 26}}):
    print(f" - {user['name']} ({user['age']})")

print("\n--- 3. Update ---")
# Update Alice's age
users.update_one({"name": "Alice"}, {"$set": {"age": 29}})
print("Updated Alice's age to 29.")

print("\n--- 4. Delete ---")
# Delete Bob
users.delete_one({"name": "Bob"})
print("Deleted Bob from the collection.")

# Count remaining
count = users.count_documents({})
print(f"\nRemaining documents in collection: {count}")

# Clean up (Optional: drop the collection to reset the demo)
# db.drop_collection("users_collection")

--- 1. Create (Insert) ---
Inserted document ID: 694ac73c623dde3fc78b65fb

--- 2. Read (Find) ---
Found Alice: {'name': 'Alice', 'age': 28, 'skills': ['Python', 'MongoDB'], '_id': ObjectId('694ac73c623dde3fc78b65fb')}
Users older than 26:
 - Alice (28)
 - Bob (32)

--- 3. Update ---
Updated Alice's age to 29.

--- 4. Delete ---
Deleted Bob from the collection.

Remaining documents in collection: 2


In [8]:
import mongomock

import pymongo
from pymongo import MongoClient
from pprint import pprint
import random
import datetime

# --- 1. Connection Setup ---
try:
    # Connect to the MongoDB client
    client = mongomock.MongoClient();

    # Create a database (if it doesn't exist)
    db = client['nosql_demo_db']

    # Create a collection (the NoSQL equivalent of a table)
    collection = db['users']

    print("--- MongoDB Connection Successful ---")
    
except Exception as e:
    print(f"Could not connect to MongoDB: {e}")
    # Exit if connection fails
    exit()

# --- Utility Function to Print Data ---
def print_all_documents(title):
    print(f"\n--- {title} ---")
    for doc in collection.find():
        pprint(doc)
    print("-" * 30)

# Clear previous data for a clean demo
collection.delete_many({})

# ==============================================================================
# 2. CREATE Operation (Insert Documents)
# ==============================================================================

# 2a. Insert One Document (Schema-less nature demo)
user_data_1 = {
    "user_id": 101,
    "username": "alice_docs",
    "email": "alice@example.com",
    "roles": ["admin", "developer"],
    "created_at": datetime.datetime.now(),
    "preferences": {
        "theme": "dark",
        "notifications": True
    }
}
insert_result_1 = collection.insert_one(user_data_1)
print(f"\nCreated 1 document. ID: {insert_result_1.inserted_id}")


# 2b. Insert Multiple Documents (Demonstrating flexible schema)
# Note how 'bob' has an 'age' field and 'charlie' has a 'status' field,
# but neither has a 'preferences' field, demonstrating schema flexibility.
user_data_2 = [
    {
        "user_id": 102,
        "username": "bob_flex",
        "age": 35,
        "email": "bob@example.com",
        "roles": ["user"],
        "last_login": datetime.datetime.utcnow()
    },
    {
        "user_id": 103,
        "username": "charlie_status",
        "status": "active",
        "roles": ["guest"],
        "favorite_color": "blue"
    }
]
collection.insert_many(user_data_2)
print(f"Created {len(user_data_2)} more documents.")

print_all_documents("Initial State: All Documents")

# ==============================================================================
# 3. READ Operation (Retrieve Documents)
# ==============================================================================

# 3a. Find One Document
print("\n--- READ 3a: Find One (user_id: 101) ---")
alice = collection.find_one({"user_id": 101})
pprint(alice)

# 3b. Find Multiple Documents (Querying by a field value)
print("\n--- READ 3b: Find Users with role 'developer' ---")
developers = collection.find({"roles": "developer"})
for dev in developers:
    pprint(dev)

# 3c. Querying a Nested Document Field
print("\n--- READ 3c: Find Users with dark theme preference ---")
dark_theme_users = collection.find({"preferences.theme": "dark"})
for user in dark_theme_users:
    print(f"User: {user['username']} - Theme: {user['preferences']['theme']}")

# 3d. Find with Condition (Using MongoDB operators like $gt)
print("\n--- READ 3d: Find Users older than 30 ($gt operator) ---")
older_users = collection.find({"age": {"$gt": 30}})
for user in older_users:
    print(f"User: {user['username']} - Age: {user['age']}")

# ==============================================================================
# 4. UPDATE Operation
# ==============================================================================

# 4a. Update One Document (Adding a new field)
# Use $set to add or update a field
update_result_1 = collection.update_one(
    {"username": "bob_flex"},
    {"$set": {"is_vip": True}}
)
print(f"\nUpdated Bob. Matched: {update_result_1.matched_count}, Modified: {update_result_1.modified_count}")

# 4b. Update Multiple Documents (Using $addToSet for Array fields)
# $addToSet adds an item to an array only if it's not already present
update_result_2 = collection.update_many(
    {"roles": "user"},
    {"$addToSet": {"roles": "beta_tester"}}
)
print(f"Updated all 'user' roles. Matched: {update_result_2.matched_count}, Modified: {update_result_2.modified_count}")

print_all_documents("State After Update Operations")

# ==============================================================================
# 5. DELETE Operation
# ==============================================================================

# 5a. Delete One Document
# Delete the user with the 'status' field
delete_result_1 = collection.delete_one({"status": "active"})
print(f"\nDeleted {delete_result_1.deleted_count} document (charlie_status).")

# 5b. Delete All Remaining Documents
delete_result_2 = collection.delete_many({})
print(f"Deleted {delete_result_2.deleted_count} remaining documents.")

print_all_documents("Final State: Collection Cleared")

# --- 6. Cleanup ---
client.close()
print("\n--- MongoDB Connection Closed ---")

--- MongoDB Connection Successful ---

Created 1 document. ID: 694ac73f623dde3fc78b65fe
Created 2 more documents.

--- Initial State: All Documents ---
{'_id': ObjectId('694ac73f623dde3fc78b65fe'),
 'created_at': datetime.datetime(2025, 12, 23, 8, 45, 51, 753000),
 'email': 'alice@example.com',
 'preferences': {'notifications': True, 'theme': 'dark'},
 'roles': ['admin', 'developer'],
 'user_id': 101,
 'username': 'alice_docs'}
{'_id': ObjectId('694ac73f623dde3fc78b65ff'),
 'age': 35,
 'email': 'bob@example.com',
 'last_login': datetime.datetime(2025, 12, 23, 16, 45, 51, 754000),
 'roles': ['user'],
 'user_id': 102,
 'username': 'bob_flex'}
{'_id': ObjectId('694ac73f623dde3fc78b6600'),
 'favorite_color': 'blue',
 'roles': ['guest'],
 'status': 'active',
 'user_id': 103,
 'username': 'charlie_status'}
------------------------------

--- READ 3a: Find One (user_id: 101) ---
{'_id': ObjectId('694ac73f623dde3fc78b65fe'),
 'created_at': datetime.datetime(2025, 12, 23, 8, 45, 51, 753000),
 '

/var/folders/hg/qd18ch2926g6xmmp0vk0lpzm0000gn/T/ipykernel_39055/2818404523.py:69: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "last_login": datetime.datetime.utcnow()


In [1]:
# docker run --name my-mongo -d -p 27017:27017 mongo:latest

from pymongo import MongoClient
from bson.objectid import ObjectId

# 1. Connect to the MongoDB Client
# If using MongoDB Atlas, replace with your connection string
client = MongoClient("mongodb://localhost:27017/")

# 2. Create/Select a Database and Collection
db = client["demo_database"]
users = db["users_collection"]

print("--- 1. Create (Insert) ---")
# Insert one document
user_data = {"name": "Alice", "age": 28, "skills": ["Python", "MongoDB"]}
result = users.insert_one(user_data)
print(f"Inserted document ID: {result.inserted_id}")

# Insert many documents
multiple_users = [
    {"name": "Bob", "age": 32, "skills": ["Java"]},
    {"name": "Charlie", "age": 25, "skills": ["Python", "JS"]}
]
users.insert_many(multiple_users)

print("\n--- 2. Read (Find) ---")
# Find a single document
alice = users.find_one({"name": "Alice"})
print(f"Found Alice: {alice}")

# Find multiple documents using a query (e.g., age > 26)
print("Users older than 26:")
for user in users.find({"age": {"$gt": 26}}):
    print(f" - {user['name']} ({user['age']})")

print("\n--- 3. Update ---")
# Update Alice's age
users.update_one({"name": "Alice"}, {"$set": {"age": 29}})
print("Updated Alice's age to 29.")

print("\n--- 4. Delete ---")
# Delete Bob
users.delete_one({"name": "Bob"})
print("Deleted Bob from the collection.")

# Count remaining
count = users.count_documents({})
print(f"\nRemaining documents in collection: {count}")

# Clean up (Optional: drop the collection to reset the demo)
# db.drop_collection("users_collection")

--- 1. Create (Insert) ---
Inserted document ID: 694f17439fda5613578ba10d

--- 2. Read (Find) ---
Found Alice: {'_id': ObjectId('694f17439fda5613578ba10d'), 'name': 'Alice', 'age': 28, 'skills': ['Python', 'MongoDB']}
Users older than 26:
 - Alice (28)
 - Bob (32)

--- 3. Update ---
Updated Alice's age to 29.

--- 4. Delete ---
Deleted Bob from the collection.

Remaining documents in collection: 2


In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import random
import datetime

# --- 1. Connection Setup ---
try:
    # Connect to the MongoDB client
    # Replace the connection string with your actual URI if not local
    client = MongoClient('mongodb://localhost:27017/')

    # Create a database (if it doesn't exist)
    db = client['nosql_demo_db']

    # Create a collection (the NoSQL equivalent of a table)
    collection = db['users']

    print("--- MongoDB Connection Successful ---")
    
except Exception as e:
    print(f"Could not connect to MongoDB: {e}")
    # Exit if connection fails
    exit()

# --- Utility Function to Print Data ---
def print_all_documents(title):
    print(f"\n--- {title} ---")
    for doc in collection.find():
        pprint(doc)
    print("-" * 30)

# Clear previous data for a clean demo
collection.delete_many({})

# ==============================================================================
# 2. CREATE Operation (Insert Documents)
# ==============================================================================

# 2a. Insert One Document (Schema-less nature demo)
user_data_1 = {
    "user_id": 101,
    "username": "alice_docs",
    "email": "alice@example.com",
    "roles": ["admin", "developer"],
    "created_at": datetime.datetime.now(),
    "preferences": {
        "theme": "dark",
        "notifications": True
    }
}
insert_result_1 = collection.insert_one(user_data_1)
print(f"\nCreated 1 document. ID: {insert_result_1.inserted_id}")


# 2b. Insert Multiple Documents (Demonstrating flexible schema)
# Note how 'bob' has an 'age' field and 'charlie' has a 'status' field,
# but neither has a 'preferences' field, demonstrating schema flexibility.
user_data_2 = [
    {
        "user_id": 102,
        "username": "bob_flex",
        "age": 35,
        "email": "bob@example.com",
        "roles": ["user"],
        "last_login": datetime.datetime.utcnow()
    },
    {
        "user_id": 103,
        "username": "charlie_status",
        "status": "active",
        "roles": ["guest"],
        "favorite_color": "blue"
    }
]
collection.insert_many(user_data_2)
print(f"Created {len(user_data_2)} more documents.")

print_all_documents("Initial State: All Documents")

# ==============================================================================
# 3. READ Operation (Retrieve Documents)
# ==============================================================================

# 3a. Find One Document
print("\n--- READ 3a: Find One (user_id: 101) ---")
alice = collection.find_one({"user_id": 101})
pprint(alice)

# 3b. Find Multiple Documents (Querying by a field value)
print("\n--- READ 3b: Find Users with role 'developer' ---")
developers = collection.find({"roles": "developer"})
for dev in developers:
    pprint(dev)

# 3c. Querying a Nested Document Field
print("\n--- READ 3c: Find Users with dark theme preference ---")
dark_theme_users = collection.find({"preferences.theme": "dark"})
for user in dark_theme_users:
    print(f"User: {user['username']} - Theme: {user['preferences']['theme']}")

# 3d. Find with Condition (Using MongoDB operators like $gt)
print("\n--- READ 3d: Find Users older than 30 ($gt operator) ---")
older_users = collection.find({"age": {"$gt": 30}})
for user in older_users:
    print(f"User: {user['username']} - Age: {user['age']}")

# ==============================================================================
# 4. UPDATE Operation
# ==============================================================================

# 4a. Update One Document (Adding a new field)
# Use $set to add or update a field
update_result_1 = collection.update_one(
    {"username": "bob_flex"},
    {"$set": {"is_vip": True}}
)
print(f"\nUpdated Bob. Matched: {update_result_1.matched_count}, Modified: {update_result_1.modified_count}")

# 4b. Update Multiple Documents (Using $addToSet for Array fields)
# $addToSet adds an item to an array only if it's not already present
update_result_2 = collection.update_many(
    {"roles": "user"},
    {"$addToSet": {"roles": "beta_tester"}}
)
print(f"Updated all 'user' roles. Matched: {update_result_2.matched_count}, Modified: {update_result_2.modified_count}")

print_all_documents("State After Update Operations")

# ==============================================================================
# 5. DELETE Operation
# ==============================================================================

# 5a. Delete One Document
# Delete the user with the 'status' field
delete_result_1 = collection.delete_one({"status": "active"})
print(f"\nDeleted {delete_result_1.deleted_count} document (charlie_status).")

# 5b. Delete All Remaining Documents
delete_result_2 = collection.delete_many({})
print(f"Deleted {delete_result_2.deleted_count} remaining documents.")

print_all_documents("Final State: Collection Cleared")

# --- 6. Cleanup ---
client.close()
print("\n--- MongoDB Connection Closed ---")

--- MongoDB Connection Successful ---

Created 1 document. ID: 694f175f9fda5613578ba111
Created 2 more documents.

--- Initial State: All Documents ---
{'_id': ObjectId('694f175f9fda5613578ba111'),
 'created_at': datetime.datetime(2025, 12, 26, 15, 16, 47, 579000),
 'email': 'alice@example.com',
 'preferences': {'notifications': True, 'theme': 'dark'},
 'roles': ['admin', 'developer'],
 'user_id': 101,
 'username': 'alice_docs'}
{'_id': ObjectId('694f175f9fda5613578ba112'),
 'age': 35,
 'email': 'bob@example.com',
 'last_login': datetime.datetime(2025, 12, 26, 23, 16, 47, 590000),
 'roles': ['user'],
 'user_id': 102,
 'username': 'bob_flex'}
{'_id': ObjectId('694f175f9fda5613578ba113'),
 'favorite_color': 'blue',
 'roles': ['guest'],
 'status': 'active',
 'user_id': 103,
 'username': 'charlie_status'}
------------------------------

--- READ 3a: Find One (user_id: 101) ---
{'_id': ObjectId('694f175f9fda5613578ba111'),
 'created_at': datetime.datetime(2025, 12, 26, 15, 16, 47, 579000),


/var/folders/hg/qd18ch2926g6xmmp0vk0lpzm0000gn/T/ipykernel_3117/2679897583.py:66: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "last_login": datetime.datetime.utcnow()
